# Install & import necessary packages

In [2]:
# !pip install langchain
# !pip install PyPDF2
# !pip install openai
# !pip install tiktoken
# !pip install InstructorEmbedding
# !pip install sentence_transformers
# !pip install faiss-cpu
# !pip install text_generation
# !pip install  google-generativeai
# !pip install langchain-google-genai
# !pip install python-dotenv
# !pip install chromadb

In [1]:
# from dotenv import load_dotenv
# load_dotenv()

In [2]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from PyPDF2 import PdfReader



import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

import faiss
from langchain.vectorstores import FAISS

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os


C:\Users\arunc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configure API keys

In [ ]:
gemini_api_key = 'YOUR_API_KEY'
genai.configure(api_key= gemini_api_key)

# Loading the pdf file and making knowledge DB

In [3]:
pdf_reader = PdfReader('48lawsofpower.pdf')
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,
            chunk_overlap=300,
            # length_function=len
            )
chunks = text_splitter.split_text(text=text)

In [5]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
VectorStore = FAISS.from_texts(chunks, embedding=embeddings)

# Init Gemini through langchain

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

model_name = "gemini-pro"
m = ChatGoogleGenerativeAI(model=model_name,convert_system_message_to_human=True, google_api_key= gemini_api_key)


In [12]:

warnings.filterwarnings("ignore")

prompt_template = """Answer the question according to the provided context. \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

stuff_chain = load_qa_chain(m, chain_type="stuff", prompt=prompt)

In [13]:
question = "Can you give me an example from history where the enemy was crushed totally from the book?"

docs = VectorStore.similarity_search(query=question, k=2)

max_attempts = 4
successful_run = False
attempts = 0

while not successful_run and attempts < max_attempts:
    try:
        stuff_answer = stuff_chain(
            {"input_documents": docs, "question": question},
            return_only_outputs=True
        )
        successful_run = True
    except Exception as e:
        print(f"Attempt {attempts + 1} failed. Error: {e}")
        attempts += 1

if successful_run:
    print(stuff_answer)
else:
    print("An error occured, please retry!")


Attempt 1 failed. Error: index: 0
finish_reason: RECITATION

{'output_text': 'The passage mentions Empress Wu of China as an example of someone who crushed her enemies totally. She used various tactics such as seduction, murder, and manipulation to eliminate anyone who challenged her throne. She ruled until the age of eighty when she was forced to abdicate, but she had successfully crushed all of her enemies during her reign.'}
